<a href="https://colab.research.google.com/github/kenia131/geeBfastMonitor/blob/master/Save_Mosaics_SC_TO_GO_allyears.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create and export rice mosaic to use in Unet
## import packges

In [1]:
# system modules
import json
import pandas as pd
import sys, os
import ee
import ee, folium
from folium import plugins

## Authenticate and initialize gee

In [2]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=J9TqWtqcDT2yvvPlMG1A35uhHeznzH4V4e6PDPmaq6s&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g5Sf4UlhBFTc1mdCrV3r1zL8ACPw3xFTaURBmn_3dnb_uDklER19zQ

Successfully saved authorization token.


In [3]:
ee.Initialize()

# Utils

## define functions

In [4]:
def getCeusius(image):
    tirC = image.select('tir1').subtract(273.5).divide(100).rename('TIR1_c')
    return image.addBands(srcImg=tirC, overwrite=True)

def getEVI2(image):
    exp = '2.5 * (b("nir") - b("red")) / (b("nir") + (2.4 * b("red")) + 1)'
    evi2 = image.expression(exp).rename(["evi2"])
    return image.addBands(srcImg=evi2, overwrite=True)

def getNDWI(image):
    exp = 'float(b("nir") - b("swir1"))/(b("nir") + b("swir1"))'
    ndwi = image.expression(exp).rename(["ndwi"])
    return image.addBands(srcImg=ndwi, overwrite=True)




## add custom basemaps to folium

In [5]:
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}


# Define a method for displaying Earth Engine image tiles on a folium map.
def addLayer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.addLayer = addLayer

# Configure

##Settings

In [34]:
ESTADOS = ee.FeatureCollection('users/agrosatelite_mapbiomas/REGIONS/ibge_estados_2019')

GRID_RICE = "users/testesMapBiomas/Teste_Arroz/GRID_ARROZ_2"

REFERENCE_MAP_RICE = "users/agrosatelite_mapbiomas/REFERENCE_MAPS/AGRICULTURE/TEMPORARY_CROPS/RICE/COLLECTION_30M"

UF = 'TO'


#CONFIGURE DATE
startDateWet = '2018-10-01'
endDateWet = '2019-04-30'
startDateDry = '2018-04-01'
endDateDry = '2018-07-30'

## Define region

In [35]:
region = ESTADOS.filter(\
                ee.Filter.Or(\
                ee.Filter.equals('SIGLA_UF', UF),\
                #ee.Filter.equals('SIGLA_UF', 'PR')\
                ))\


grid = ee.FeatureCollection(GRID_RICE).filterBounds(region.geometry())
          # .filterMetadata('RICE_TRAIN', 'equals', 1)
    
mask = ee.Image(1).clip(grid)      

arroz = ee.ImageCollection(REFERENCE_MAP_RICE)\
            .filterMetadata('year', 'equals', YEAR)\
            .mosaic()                 

## Define cloud mask

In [36]:
bandNames = ee.Dictionary({
  'LANDSAT_5': ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'B6', 'BQA'],
  'LANDSAT_7': ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'B6_VCID_1', 'BQA'],
  'LANDSAT_8': ['B2', 'B3','B4', 'B5', 'B6', 'B7', 'B10', 'BQA']
});

def padronizeBandNames (image):
  oldBandNames = bandNames.get(image.get('SPACECRAFT_ID'))
  newBandNames = ['blue', 'green', 'red',  'nir', 'swir1', 'swir2', 'tir1', 'BQA']
  return image.select(oldBandNames, newBandNames)

qaBits57 = ee.List([
  [0, 0, 0], #  Designated Fill
  [1, 1, 0], # Designated Pixel
  [4, 4, 0], # cloud-free
  [7, 8, 1]  # Cloud Shadow Confidence is Low
]);

qaBits8 = ee.List([
  [0, 0, 0],
  [1, 1, 0],
  [4, 4, 0],
  [5, 6, 1],
  [7, 8, 1],
  [11, 12, 1]
]);


qaBitsDict = ee.Dictionary({
  'LANDSAT_5': qaBits57,
  'LANDSAT_7': qaBits57,
  'LANDSAT_8': qaBits8
});

def itFunc (i, pattern):
      i = ee.Number(i)
      pattern = ee.Number(pattern);

      return pattern.add(ee.Number(2).pow(i));
    

def getQABits(image, start, end):
    pattern = ee.Number(ee.List.sequence(start, end).distinct().iterate(itFunc, ee.Number(0)));

    return image.select(0).bitwiseAnd(pattern.int()).rightShift(start);


def maskClouds(image):
    qaBits = ee.List(qaBitsDict.get(image.getString('SPACECRAFT_ID')));
    bqa = image.select('BQA');


    inital_state = ee.Dictionary({
        'bqa': bqa,
        'mask': ee.Image(1)});
  
    def it2 (bits, state):
        bits = ee.List(bits);
        state = ee.Dictionary(state);

        bqa = ee.Image(state.get('bqa'));
        mask = ee.Image(state.get('mask'));

        start = bits.getNumber(0);
        end = bits.getNumber(1);
        desired = bits.getNumber(2);

        blueprint = getQABits(bqa, start, end).eq(desired);

        return ee.Dictionary({
            'bqa': bqa,
            'mask': mask.updateMask(blueprint)
        });
    
  
    
    finalState = ee.Dictionary(qaBits.iterate(it2, inital_state));
  
    cloudMask = ee.Image(finalState.get('mask'));

    return image.updateMask(cloudMask);

## create filtered Landsat collection

In [37]:
l5Collection = ee.ImageCollection("LANDSAT/LT05/C01/T1_TOA");
l7Collection1 = ee.ImageCollection("LANDSAT/LE07/C01/T1_TOA");
l7Collection2 = ee.ImageCollection("LANDSAT/LE07/C01/T1_TOA");
l8Collection = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA");

collection_filtered = l8Collection.merge(l5Collection).merge(l7Collection1).merge(l7Collection2)\
    .map(padronizeBandNames)\
    .map(maskClouds)\
    .map(getNDWI)\
    .map(getEVI2)\
    .map(getCeusius)\
    .filterBounds(region)\
   


## Define Season and offSeason periods

In [38]:
wet = collection_filtered.filterDate(startDateWet,endDateWet)\
              .qualityMosaic('evi2')
          
          
wetNames = wet.bandNames().map(lambda band: ee.String(band).cat('_wet'))
wet = wet.rename(wetNames)

dry = collection_filtered.filterDate(startDateDry,endDateDry).min()
          
           
dryNames = dry.bandNames().map(lambda band: ee.String(band).cat('_dry'))
dry = dry.rename(dryNames)

                                      
mosaic = wet.addBands(dry)

## calculate CEI from EVI2

In [39]:

cei   = mosaic.expression('100*(WET_max - DRY_min) / (100+WET_max + 100+DRY_min)', {
  'WET_max': mosaic.select(['evi2_wet', 'ndwi_wet']),
  'DRY_min': mosaic.select(['evi2_dry', 'ndwi_dry']),
}).rename(['cei_evi2', 'cei_ndwi'])


mosaic = mosaic.addBands(cei)

mosaicUnet = mosaic.select(['cei_evi2', 'cei_ndwi', 'swir2_dry']).multiply(255).int16().updateMask(mask) 

In [40]:
print (mosaicUnet.bandNames().getInfo())

['cei_evi2', 'cei_ndwi', 'swir2_dry']


## Visualize mosaic

In [41]:
# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 80,
  'bands': ['cei_evi2', 'cei_ndwi', 'swir2_dry']}

# Create a folium map object.
my_map = folium.Map(location=[20, 0], zoom_start=3, height=500)

# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)

# Add the mosaic to the map object.
my_map.addLayer(mosaicUnet, vis_params, 'mosaicUnet')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
display(my_map)

## Export Tasks

In [42]:
list2 = grid.aggregate_array("id").distinct().sort()
print(list2.length().getInfo())

8


In [43]:
for i in range(list2.length().getInfo()):
    roi = grid.filterMetadata('id', 'equals', list2.get(i))
    mosaicExp = mosaicUnet

    mosaic_toExport = mosaicExp.unmask().clip(roi)
    label_toExport = arroz.unmask().clip(roi).byte()

    print ("export mosaic {}_{}".format(str((list2.get(i)).getInfo()), str(YEAR)))

    task = ee.batch.Export.image.toDrive(
        image=mosaic_toExport,
        description='id_' + str((list2.get(i)).getInfo()) +'_'  + 'mosaic_' + str(YEAR),
        folder= 'TRAIN LANDSAT ARROZ_pythonVersion_{}_{}'.format(str(YEAR), str(UF)),
        fileNamePrefix= 'id_' + str((list2.get(i)).getInfo()) + '_'  + 'mosaic',
        region=roi.geometry(),
        scale=30,
        crs= 'EPSG:3857',
        maxPixels=int(1e13),
        shardSize= 32,
        )

    task.start()    

export mosaic 769_2018
export mosaic 770_2018
export mosaic 771_2018
export mosaic 772_2018
export mosaic 820_2018
export mosaic 821_2018
export mosaic 822_2018
export mosaic 919_2018
